# System Call Process Demo

This interactive demonstration illustrates the implementation of a system call.  
While it is based on the xv6 operating system, it uses Python to simplify the concepts and enhance understanding.

---



### **What to Know**

**Conceptual Overview**

To simplify a bit, the steps an OS will take to perform a system call are:
1. User Mode
    - The user space initiates a system call using a wrapper function provided by a library. For C lang, that would be `glibc`
2. Transition to Kernel Mode
    - The OS performs a context switch to temporarily change from user mode to kernel mode.
3. Kernel Mode 
    - This mode gives us full control over hardware and system resources. Enabling us to perform critical tasks and manage the system.
4. Execution of the System Call
    - Kernel identifies the syscall and executes the corresponding function to perform the operation.
5. Transition back to User Mode
    - OS performs another context switch to return from kernel mode to user mode, passing the result back to the user application.  
    
<hr style="border: none; border-top: 1px dashed #ccc;" />


**How a Syscall is Defined**

In order to define a system call to the operating system, we need multiple files which will all work together to tell the OS *exactly* what it needs to perform properly.
1. `user.h`
    - Declares user-space functions for making syscalls. It contains declarations of functions (wrappers) that user-space programs can call to perform system calls.
2. `usys.S`
    - Assembly code that will select the correct syscall for the OS to use. (The one which the user selected)
3. `syscall.h`
    - Defines the unique numbers for each system call. These numbers are used by the user-space program and the system call interface (usys.S) to identify which system call to execute. 
4. `syscall.c`
    - Dispatches the syscall using definitions and tables to read the syscall number from the process's register, looks up the corresponding handler in the table, then executes the handler (if valid) or returns an error.
5. `sysproc.c`
    - Contains the implementation of the actual system call handler functions. It defines what each system call should do when invoked.
---



### **Imports**

In [ ]:
import time, sys

### **Implementing System Calls**

**SYSCALL.H**

Let's start by going in our "operating system"'s syscall.h "file" and make some constants and assign them their system call numbers:

In [ ]:
# syscall.h
SYSCALL_ECHO = 0
SYSCALL_TIME = 1

time.sleep(1); print("\033[92mReading System Call Numbers...\033[0m")

**SYSPROC.C**

Now we take a look at our sysproc.c "file" and add our implementation of our custom system call.

In [ ]:
# sysproc.c: Kernel functions
def kernel_echo(message):
    sys.stdout.write(f"{message}\n")
    sys.stdout.flush() # Ensure message prints immediately

def kernel_time():
    return time.time()

# sysproc.c: Syscall table
syscall_table = {
    SYSCALL_ECHO: kernel_echo,
    SYSCALL_TIME: kernel_time
}

print("\033[92mReferencing definitions and table to find system call function...\033[0m")

**SYSCALL.C**

The system call interface for the OS

In [ ]:
# syscall.c: Syscall interface
def syscall(syscall_number, *args):
    # Transition to Kernel Mode
    time.sleep(2); print(f"\033[92mTransition: Switching to Kernel Mode for syscall {syscall_number}\033[0m")
    
    # Simulate the kernel's handling of the syscall
    if syscall_number in syscall_table:
        result = syscall_table[syscall_number](*args)
        
        # Return to User Mode
        time.sleep(2); print("\033[92mReturn: Switching back to User Mode\033[0m")
        return result
    else:
        raise ValueError("Invalid syscall number")

print(f"\033[92mLoading system call interface...\033[0m")

**USYS.S**

Now we edit the assembly code that helps the OS bridge together the high level definitions of our system call to the low-level instructions that we need to execute.

In [ ]:
# usys.S: Low-level syscall interface (simulated in Python)
def low_level_syscall(syscall_number, *args):
    time.sleep(2); print(f"\033[92mInitializing low-level system call handler (asm) ...\033[0m")

    # NOTE This function would actually use assembly in a real OS
    return syscall(syscall_number, *args)

print(f"\033[92mInitializing low-level system call handler (asm) ...\033[0m")

**USER.H**

Contains the user level system call definitions. What we see when *we* use the syscalls.

In [ ]:
# user.h: User program interface
def echo(message):
    low_level_syscall(SYSCALL_ECHO, message)

def get_time():
    return low_level_syscall(SYSCALL_TIME)

print(f"\033[92mReading user level system call definitions...\033[0m")

### **MAIN.C** (Our program)
"Compile" and run the OS, and use our custom SYSCALLS!

Remember: in reality, the calls will be super fast. I've slowed them down to make it easier to see whats happening.

In [7]:
# MAIN.C: Main execution
def main():
    while True:
        cmd = input("OS >> ")
        if cmd == "exit":
            break
        elif cmd == "time":
            print(get_time())
        elif cmd.startswith("echo "):
            echo(cmd[5:])
        else:
            print("Unknown command")

if __name__ == "__main__":
    main()